# Generador de Informe de Salarios

## Visión General

Este script automatiza la creación de un informe de salarios utilizando Google Sheets. El proceso incluye:

1. Generación de datos de empleados de muestra.
2. Creación de una nueva hoja de cálculo en Google Sheets.
3. Procesamiento de los datos de empleados (cálculo de salario anual y años en la empresa).
4. Escritura de los datos procesados en la hoja de Google Sheets.
5. Compartir la hoja de cálculo con un correo electrónico específico.

Funciones principales:
- `generar_datos_empleados`: Crea datos de muestra para los empleados.
- `crear_hoja_google`: Crea una nueva hoja de cálculo en Google Sheets.
- `compartir_hoja_google`: Comparte la hoja de cálculo con un correo electrónico.
- `procesar_datos_empleados`: Calcula información adicional sobre los empleados.
- `escribir_en_google_sheets`: Escribe los datos procesados en la hoja de Google Sheets.

Para utilizar este script:
1. Asegúrate de tener instaladas las bibliotecas necesarias (gspread, pandas, google-auth, etc.).
2. Coloca el archivo "credentials.json" con tus credenciales de la API de Google Sheets en el mismo directorio.
3. Ajusta la variable `correo_personal` en la función `main()` con tu dirección de correo electrónico.
4. Ejecuta el script para generar y compartir el informe de salarios.

In [53]:
# Generador de Informe de Salarios

## 1. Importar bibliotecas necesarias

import gspread
import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from datetime import datetime
import random
from faker import Faker

## 2. Configurar autenticación de Google Sheets

def obtener_autenticacion_google():
    """
    Configura y retorna las credenciales para la API de Google.
    
    Returns:
        Credentials: Objeto de credenciales para autenticar con la API de Google.
    """
    ALCANCE = [
        "https://spreadsheets.google.com/feeds",
        "https://www.googleapis.com/auth/spreadsheets",
        "https://www.googleapis.com/auth/drive.file",
        "https://www.googleapis.com/auth/drive"
    ]
    credenciales = Credentials.from_service_account_file("credentials.json", scopes=ALCANCE)
    return credenciales

## 3. Operaciones de Google Sheets

def crear_hoja_google(nombre_hoja):
    """
    Crea una nueva hoja de cálculo en Google Sheets con una sola hoja de trabajo.
    
    Args:
        nombre_hoja (str): Nombre de la nueva hoja de cálculo.
    
    Returns:
        tuple: ID de la hoja de cálculo creada y el objeto de la hoja de trabajo.
    """
    credenciales = obtener_autenticacion_google()
    cliente = gspread.authorize(credenciales)
    
    # Crear una nueva hoja de cálculo
    hoja_calculo = cliente.create(nombre_hoja)
    id_hoja = hoja_calculo.id
    
    # Renombrar la primera hoja (por defecto 'Sheet1') a 'Reporte de Salarios'
    hoja_trabajo = hoja_calculo.get_worksheet(0)
    hoja_trabajo.update_title("Reporte de Salarios")
    
    print(f"Hoja de cálculo creada con ID: {id_hoja}")
    return id_hoja, hoja_trabajo

def compartir_hoja_google(id_hoja, email):
    """
    Comparte la hoja de cálculo con un correo electrónico específico.
    
    Args:
        id_hoja (str): ID de la hoja de cálculo a compartir.
        email (str): Dirección de correo electrónico con la que se compartirá.
    """
    credenciales = obtener_autenticacion_google()
    cliente = gspread.authorize(credenciales)
    hoja_calculo = cliente.open_by_key(id_hoja)
    hoja_calculo.share(email, perm_type='user', role='writer')
    print(f"Hoja de cálculo compartida con {email}")

def escribir_en_google_sheets(df, hoja_trabajo):
    """
    Escribe un DataFrame en una hoja de trabajo de Google Sheets.
    
    Args:
        df (DataFrame): DataFrame de pandas a escribir.
        hoja_trabajo (Worksheet): Objeto de hoja de trabajo de gspread.
    """
    hoja_trabajo.clear()
    hoja_trabajo.update([df.columns.values.tolist()] + df.values.tolist())
    print("Datos escritos en la hoja de cálculo.")

def cambiar_nombre_hoja_calculo(id_hoja, nuevo_nombre):
    """
    Cambia el nombre de una hoja de cálculo de Google Sheets.
    
    Args:
        id_hoja (str): ID de la hoja de cálculo.
        nuevo_nombre (str): Nuevo nombre para la hoja de cálculo.
    """
    credenciales = obtener_autenticacion_google()
    cliente = gspread.authorize(credenciales)
    hoja_calculo = cliente.open_by_key(id_hoja)
    hoja_calculo.update_title(nuevo_nombre)
    print(f"Hoja de cálculo renombrada a: {nuevo_nombre}")

## 4. Generación y procesamiento de datos

def generar_datos_empleados(num_empleados=10):
    """
    Genera datos de empleados de muestra.
    
    Args:
        num_empleados (int): Número de empleados a generar (por defecto 10).
    
    Returns:
        DataFrame: DataFrame de pandas con los datos de los empleados.
    """
    fake = Faker()
    datos_empleados = []
    
    for _ in range(num_empleados):
        nombre = fake.name()
        salario = round(random.uniform(2000, 8000), 2)
        fecha_contratacion = fake.date_between(start_date='-24y', end_date='today')
        
        datos_empleados.append({
            "Nombre": nombre,
            "Salario Mensual": salario,
            "Fecha de Contratación": fecha_contratacion
        })
    
    return pd.DataFrame(datos_empleados)

def procesar_datos_empleados(df):
    """
    Procesa los datos de los empleados, calculando salario anual y años en la empresa.
    
    Args:
        df (DataFrame): DataFrame de pandas con los datos originales de los empleados.
    
    Returns:
        DataFrame: DataFrame de pandas con los datos procesados.
    """
    df['Salario Mensual'] = pd.to_numeric(df['Salario Mensual'], errors='coerce')
    df['Fecha de Contratación'] = pd.to_datetime(df['Fecha de Contratación'])
    
    df['Salario Anual'] = df['Salario Mensual'] * 12
    
    fecha_actual = datetime.now()
    df['Años en la Empresa'] = (fecha_actual - df['Fecha de Contratación']).dt.days // 365
    
    df['Fecha de Contratación'] = df['Fecha de Contratación'].dt.strftime('%Y-%m-%d')
    
    return df[['Nombre', 'Salario Mensual', 'Salario Anual', 'Fecha de Contratación', 'Años en la Empresa']]

## 5. Ejecución principal

def main():
    """
    Función principal que ejecuta todo el proceso de generación y carga del informe de salarios.
    """
    # Generar datos de muestra
    df_empleados = generar_datos_empleados(10)
    df_empleados.to_csv('empleados.csv', index=False)
    
    # Crear y configurar la hoja de Google
    nombre_hoja = "Reporte de Salarios"
    id_hoja, hoja_trabajo = crear_hoja_google(nombre_hoja)
    correo_personal = "hectormanujuarez1987@gmail.com"  # Reemplaza con tu correo
    compartir_hoja_google(id_hoja, correo_personal)
    
    # Procesar y escribir datos
    df_procesado = procesar_datos_empleados(df_empleados)
    escribir_en_google_sheets(df_procesado, hoja_trabajo)
    
    print(f"Proceso completado. Puedes acceder a tu hoja de cálculo en: https://docs.google.com/spreadsheets/d/{id_hoja}")

if __name__ == "__main__":
    main()

Hoja de cálculo creada con ID: 1qAaXFdnEKozklJYZ7XmAu5ygd_Cw7dzAwbiLKUlLoH4
Hoja de cálculo compartida con hectormanujuarez1987@gmail.com
Datos escritos en la hoja de cálculo.
Proceso completado. Puedes acceder a tu hoja de cálculo en: https://docs.google.com/spreadsheets/d/1qAaXFdnEKozklJYZ7XmAu5ygd_Cw7dzAwbiLKUlLoH4


In [4]:
# Initialize Faker instance
fake = Faker()

# Function to generate random employee data
def generate_employee_data(num_employees=10):
    employee_data = []
    
    for _ in range(num_employees):
        # Generate a random name
        name = fake.name()
        
        # Generate a random monthly salary between $2000 and $8000
        salary = round(random.uniform(2000, 8000), 2)
        
        # Generate a random hire date between 2000-01-01 and 2023-01-01
        hire_date = fake.date_between(start_date='-24y', end_date='today')
        
        # Add the data to the list
        employee_data.append({
            "Nombre": name,
            "Salario Mensual": salary,
            "Fecha de Contratación": hire_date
        })
    
    return pd.DataFrame(employee_data)

# Generate data for 10 employees
df_employees = generate_employee_data(10)

# Show the generated data
print(df_employees)

# Save the data to a CSV file (optional)
df_employees.to_csv('empleados.csv', index=False)


              Nombre  Salario Mensual Fecha de Contratación
0        Harry Brown          6618.08            2004-12-29
1      Sierra Torres          5937.50            2008-11-22
2     Ian Pennington          4352.37            2024-08-14
3   Samantha Johnson          7026.78            2007-09-02
4  Samantha Williams          6190.42            2012-06-17
5          John Hess          4647.67            2017-03-31
6     Kristin Garcia          3295.30            2008-03-22
7      Thomas Torres          2661.85            2022-09-08
8  Christopher Wells          4195.55            2002-08-11
9      James Summers          4183.73            2007-11-24
